# Instrument Connection

In [3]:
# !pip install srsinst.sr860
# !pip install pymeasure
# !pip install lmdb
import h5py
import time
import numpy as np
import pandas as pd
import pyvisa as visa
from pprint import pprint
from typing import Dict
from datetime import datetime

from srsinst.sr860 import SR860
from pymeasure.instruments.keithley import Keithley2400

rm = visa.ResourceManager()
# Show candidates of devices
pprint(rm.list_resources())

visa_addr_top = "GPIB2::4::INSTR"
visa_addr_keithley = "GPIB2::3::INSTR"

lockin1 = SR860("visa", visa_addr_top)
gate = Keithley2400(visa_addr_keithley)

('ASRL1::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL7::INSTR',
 'ASRL8::INSTR',
 'ASRL10::INSTR',
 'ASRL15::INSTR',
 'GPIB1::3::INSTR',
 'GPIB2::3::INSTR',
 'GPIB2::4::INSTR',
 'USB0::0xB506::0x2000::005808::0::INSTR',
 'USB0::0xB506::0x2000::123456::0::INSTR')


# Instrument Connection Check

In [4]:
# Check instrument with below code

instrument = rm.open_resource(visa_addr_top)
idn_response = instrument.query("*IDN?")
print(idn_response) # Stanford_Research_Systems,SR860,005808,1.55

if not "Stanford_Research_Systems,SR860" in idn_response:
    raise Exception("It is not SR860...")
else:
    print("successfully connected to top SR860...")

    
instrument = rm.open_resource(visa_addr_keithley)
idn_response = instrument.query("*IDN?")
print(idn_response) # KEITHLEY INSTRUMENTS INC.,MODEL 2401,4631577,B02 Jan 20 2021 10:19:49/B01  /W/N

if not "KEITHLEY INSTRUMENTS INC.,MODEL 2401" in idn_response:
    raise Exception("It is not Keithley...")
else:
    print("successfully connected to Keithley...")

Stanford_Research_Systems,SR860,005808,1.55

successfully connected to top SR860...
KEITHLEY INSTRUMENTS INC.,MODEL 2401,4631577,B02 Jan 20 2021 10:19:49/B01  /W/N

successfully connected to Keithley...


# Measurement Function

In [5]:
def measurement(
    v_g: float = 0,
    lockin1: SR860 = None,
    gate: Keithley2400 = None,
) -> Dict[
    str, int
]:
    """
    Measure with lockin instrument
    """
    gate.enable_source()
    gate.ramp_to_voltage(v_g, steps = 2, pause = 0.1)
    lock_in_wait_time = 3 * 0.3
    time.sleep(lock_in_wait_time)

    x1 = lockin1.data.get_values("X","Y")[0]
    y1 = lockin1.data.get_values("X","Y")[1]
    r1 = lockin1.data.get_values("R","Theta")[0]
    theta1 = lockin1.data.get_values("R","Theta")[1]

    return {
        "X" : x1,
        "Y" : y1,
        "R" : r1,
        "angle" : theta1
    }


# Data Measurement

In [8]:
# Disable Keithley 2400 output
gate.apply_voltage()
gate.disable_source()
# Set Keithley 2400 voltage
gate.source_voltage_range = 5.0
gate.source_voltage = 0.0

# Set Keithley 2400 compliance current
gate.source_current_range = 1e-9
gate.compliance_current = 100e-9

# Setup upper lockin device
V_bias = 100e-6
time_constant = 0.3

lockin1.ref.sine_out_amplitude = V_bias
lockin1.ref.frequency = 43.5371
lockin1.ref.phase = 0
lockin1.signal.filter_slope = 24
lockin1.signal.advanced_filter = "on"
lockin1.signal.time_constant = time_constant

lock_in_wait_time = 3 * time_constant

v_g_list = np.linspace(0, 5.0, 3)

time.sleep(0.1)

data_set = {
    "X" : [],
    "Y" : [],
    "R" : [],
    "angle" : [],
}

for v_g in v_g_list:
    meas_data = measurement(
        v_g = v_g,
        lockin1 = lockin1,
        gate = gate,
    )
    data_set["X"].append([v_g, meas_data["X"]])
    data_set["Y"].append([v_g, meas_data["Y"]])
    data_set["R"].append([v_g, meas_data["R"]])
    data_set["angle"].append([v_g, meas_data["angle"]])

    time.sleep(1)


# Disable instrument outputs
gate.disable_source()
lockin1.ref.sine_out_amplitude = 0.0

naive_dt = datetime.now()
_date = naive_dt.strftime("%Y-%m-%d_%H_%M_%S")
with h5py.File(f"led_data_{_date}.h5", "w") as f:
    for key, value in data_set.items():
        if isinstance(value, dict):
            # Create a group for nested dictionaries
            group = f.create_group(key)
            for sub_key, sub_value in value.items():
                group.create_dataset(sub_key, data=sub_value)
        else:
            # Store other data types as datasets
            f.create_dataset(key, data=value)